### Librerias

## variables de carga

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

DATE_START   = os.getenv("DATE_START")      # YYYY-MM-DD
DATE_END     = os.getenv("DATE_END")        # YYYY-MM-DD

WORKDAY_START = os.getenv("WORKDAY_START")  # HH:MM
WORKDAY_END   = os.getenv("WORKDAY_END")    # HH:MM

TASK_START_TIME = os.getenv("TASK_START_TIME")  # HH:MM
TASK_END_TIME   = os.getenv("TASK_END_TIME")    # HH:MM

EVENT_NAME   = os.getenv("EVENT_NAME")
SLOT_MINUTES = int(os.getenv("SLOT_MINUTES", "30"))
EVENT_COLOR_ID = os.getenv("EVENT_COLOR_ID")

CALENDAR_ID  = os.getenv("CALENDAR_ID", "primary")
TIMEZONE     = os.getenv("TIMEZONE", "America/Bogota")

print("Rango de fechas:", DATE_START, "->", DATE_END)
print("Horario laboral:", WORKDAY_START, "-", WORKDAY_END)
print("Rango tarea:", TASK_START_TIME, "->", TASK_END_TIME)



In [ ]:
from datetime import datetime as dt, timedelta

from method.authentication import get_calendar_service
from method.generateBlocksFunction import (
    create_time_slots,
    create_event,
    filter_free_slots,
)
from method.validations import (
    _max_time_str,
    _min_time_str,
    get_busy_periods_for_day
    )

In [ ]:
service = get_calendar_service()

start_date_dt = dt.strptime(DATE_START, "%Y-%m-%d").date()
end_date_dt   = dt.strptime(DATE_END, "%Y-%m-%d").date()

current_date = start_date_dt

total_slots = 0
total_created = 0

while current_date <= end_date_dt:
    date_str = current_date.strftime("%Y-%m-%d")
    print(f"\nProcesando día {date_str}...")

    
    if start_date_dt == end_date_dt:
        day_start_str = _max_time_str(WORKDAY_START, TASK_START_TIME)
        day_end_str   = _min_time_str(WORKDAY_END, TASK_END_TIME)
  
    elif current_date == start_date_dt:
        day_start_str = _max_time_str(WORKDAY_START, TASK_START_TIME)
        day_end_str   = WORKDAY_END

    elif current_date == end_date_dt:
        day_start_str = WORKDAY_START
        day_end_str   = _min_time_str(WORKDAY_END, TASK_END_TIME)

    else:
        day_start_str = WORKDAY_START
        day_end_str   = WORKDAY_END


    if dt.strptime(day_start_str, "%H:%M") >= dt.strptime(day_end_str, "%H:%M"):
        print(f"Saltando día {date_str} porque el rango calculado {day_start_str}-{day_end_str} no es válido.")
        current_date += timedelta(days=1)
        continue


    day_slots = create_time_slots(
        date_str=date_str,
        start_str=day_start_str,
        end_str=day_end_str,
        duration_minutes=SLOT_MINUTES
    )


    busy_periods = get_busy_periods_for_day(
        service=service,
        calendar_id=CALENDAR_ID,
        date_str=date_str,
        start_time_str=day_start_str,
        end_time_str=day_end_str,
        tz=TIMEZONE,
    )


    free_slots = filter_free_slots(day_slots, busy_periods)

    print(f"Bloques generados: {len(day_slots)}  |  Bloques libres: {len(free_slots)}")


    for start_dt, end_dt in free_slots:
        create_event(
            service=service,
            calendar_id=CALENDAR_ID,
            summary=EVENT_NAME,
            start_dt=start_dt,
            end_dt=end_dt,
            tz=TIMEZONE,
            color_id=EVENT_COLOR_ID
        )
        total_created += 1

    total_slots += len(day_slots)
    current_date += timedelta(days=1)

print(f"\nResumen:")
print(f"Bloques totales generados en el rango: {total_slots}")
print(f"Eventos creados (sin sobreagendar): {total_created}")
print("Listo.")
